# 농작물 추천을 위한 수확량 회귀분석모델 개발

- date : 2024.04.13 ~
- 

In [1]:
#-------------------------------------------------------------------------------------------------------------------- # nolint
## Descrition : Library 설치및 기본 세팅 셀
## Auhtor : Forrest Park
## Update :
    #2024.04.12 Sat
        # 1. RandomForrest model 함수 추가
    #2024.04.13 Sun
        # 1. data Fetch 함수화
#--------------------------------------------------------------------------------------------------------------------
## Basic Settings
    
   
    options(warn = -1) ## 경고 지우기
    options(repr.plot.width=8, repr.plot.height=6) # 플랏 크기 조절
    Sys.setlocale(category = "LC_ALL", locale = "ko_KR.UTF-8") # 한글 설정

# Basic Graph Google Font Setting
    library(showtext) ## plot : 구글 폰트 사용 
    font_add_google(name = "Noto Serif KR",family = "noto-serif")
    showtext_auto(TRUE)
# Libraries for data fetching
    library(jsonlite) # JSON file fetch
    # fromJSON("") #: json file fetch
    library(XML) # XML file fetch
    # xmlToDataFrame("")
    library(readxl) # excel file fetch
    # read_excel("", sheet =2)
    library(data.table)
    # tidyverse 
    library(tidyverse)

## Data Fetch (csv)
    dataFetch_csv <- function(csvfile_address){
        ## Descrition : Data fetch function for csv file 
        ## Auhtor : Forrest Park
        ## Update :
        #2024.04.13 Sun
            # 1. 
        ## CSV filez
        return(read.csv(csvfile_address,na.strings=NA, header = T, sep=",",encoding = "UTF-8",fileEncoding="EUC-KR")) 
    }

## Data Fetch (txt)
    dataFetch_txt <- function(txtfile_address){
        ## Descrition : Data fetch function for txt file 
        ## Auhtor : Forrest Park
        ## Update :
        #2024.04.13 Sun
            # 1. 
        return(readtable(txtfile_address, 
                encoding ="EUC-KR", fileEncoding ="UTF-8", 
                colnames =varnames,
                hearder =F, skip =0, nrows =-1, sep=""))
    }

## DataFetch ( xlsx)
    dataFetch_xlsx <-function(xlsxfile_address){
        ## Descrition : xlsx 파일에서 데이터 읽기
        ## Auhtor : Forrest Park
        ## Update :
            #2024.04.13 Sun
                # 1. 
        return(read_excel(xlsxfile_address))
    } 
## 함수 정리 

## 기본함수 : data, str, length, ncol, nrow, dim, ls, head, tail, library, 
    # data<- c(1,2,3)
    # label = c('a','b','c')
    # names(data)<-label
    
    # matrix( data, nrow= 2, ncol=3 , byrow=T)
    # array(data, dim = c(2,3))
    # list(data, "hello")
    # factor(data, labels= label)

## 기본 통계량 mean, median, min, max, range, sd, hist, barplot, stem, var
    # range(data) #: data 의 최대 최소 출력
    # var(data)  #: data 의 분산값 출력
    # quantile(data, probs = 0.25) # :  사분위 수 출력 함수 

## 데이터 처리 : apply, sapply, lapply 

## 데이터 시각화
    # stem(col, scale =1)
    # hist(data) # : data 의 histogram(연속 분포함수 ) 
    # barplot(data, ylim =c(0:10), main= "title", xlab="x label", ylab="y label", names =colnames(data)), 


## Libraries for DATA
    library(stringr) # stringr 불러오기
    library(dplyr) # dplyr : 데이터 가공 함수 -> 
    # filter(data,  a>1) : 특정 row 추출
    # select(data, a, b) : 특정 column 추출
    # arrange(data, desc(a))
    # mutate(data, a_rank =rank(a)) # a칼럼 값을 기준으로 rank 를 매겨 새로운 랭크 칼럼을 생성
    # distinct(data, a,b) # data 의 a, b 칼럼을 기준으로 중복된 행을 제거 (a와 b 가 동시에 중복)
    # summarise(data, a_mean = mean(a), a_min= min(a), a_sd = sd(a) .. func(a)) # a 칼럼의 평균, 등등의 함수 값 요약. 

    library(descr) # descr : 데이터 설명 함수 
    # freq(data, plot =F, main ="title") # : data row 별 빈도수 
    library(psych) # psych : 데이터 시각화 함수 pairs.panel(iris)
    # kurtosi(data) #: data 의 첨도값 출력 / psych 
    # skew(data) #: data 의 왜도값 출력 /psysy


    library(reshape2) # data layout 변환 
    library(ggplot2) # 그래프 
    library(plotrix) # 3D pie
    library(RColorBrewer) # color package
    library(tidyverse)
    library(hrbrthemes)
    library(kableExtra)

    library(caret) # data sampling 
    library(randomForest) # prediction model 
    library(nnet)


#library(extrafont)
# Functions 
dataDescription <- function(data){
    columns <-c()
    cat("\n\n *** 칼럼 별 데이터 빈도 조사 ***\n\nData Columns :  ",ncol(data),"개","\n\n\n")
    for( i in 1:ncol(data)){
        cat(" ",i ,". ", colnames(data)[i],"\n", sep = "")
        columnDesc(data[colnames(data)[i]])
        columns<- append(columns,colnames(data)[i])
    }
    return(columns)
}

columnDesc<- function(column){

    ## Descrition : 칼럼 데이터 설명 
    ## Auhtor : Forrest Park
    ## Update :
        #2024.04.13 Sun
            # 1. 
    par(bg = "white")
    # 시장이름의 개수
    cat( "\n ***[",names(column), "]칼럼의 데이터 종류 개수:",nrow(unique(column)), sep = "")
    # 가장 빈도가 많은 데이터와 적은 데이터 각각 5개 
    dataFreq_top5<- head(arrange(as.data.frame(freq(as.matrix(column), plot =F)), desc(Frequency))[2:6,],5)
    dataFreq_bot5<-tail(arrange(as.data.frame(freq(as.matrix(column), plot =F)), desc(Frequency)),5)
    cat("\n\n-----[",names(column),"] 에서 가장 빈도가 많은 데이터 top 5 \n")
    print(dataFreq_top5)
    cat("\n\n-----[",names(column),"] 에서 가장 빈도가 적은 데이터 5 개 \n")
    print(dataFreq_bot5)

    cat("\n\n\n")

}

## 특정칼럼의 데이터를 모두 보고싶을 때 csv 로 변환하는 함수 

column_csvSaving <- function(column_vector,title){
    ##library(data.table)
    ##library(tidyverse)
    a_data<-as.data.frame(data.table(distinct(column_vector)))
    write_csv(a_data,paste(title,".csv",sep =""))
}


row_kind_show<-function(data){
    ## Descrition : 각 칼럼에서의 값들이 어떤 것들이있는지 개수를 알려줌. 
    ## Auhtor : Forrest Park
    ## Update :
        #2024.04.13 Sun
            # 1. 
    cols <- colnames(data)
    for ( i in 1:length(cols)){
        cat(paste(cols[i],'의 중복되지않은 개수 : ',length(unique(data[[cols[i]]])) ,sep=""  ), "\n")
    }
}


findWord_in_specific_colum <-function(data,찾을칼럼이름, 찾아야하는단어){
    ## Descrition : 특정칼럼에서 어떤단어를 포함하는 row 만 골라내기
    ## Auhtor : Forrest Park
    ## Update :
        #2024.04.13 Sun
            # 1. 
    col_num<-which(colnames(data)==찾을칼럼이름)
    selectedCol <- data[[찾을칼럼이름]] 
    data %>% filter(str_detect(selectedCol,찾아야하는단어))
}

#a %>% filter(VGETBL_CL=='수박')
# a_data<-as.data.frame(data.table(distinct(채소생산['VGETBL_CL'])))
# write_csv(a_data,"a_data.csv")



ERROR: Error in parse(text = x, srcfile = src): <text>:165:44: unexpected '<'
164: 
165: findWord_in_specific_colum <-function(data,<
                                                ^
